In [1]:
import pandas as pd
import numpy as np
import math as m
from itertools import combinations

DATA_PATH = './data/Election results.xlsx'

Для начала загрузим табличку с данными, которую я собрал по выборам в парламент греции в 2019 году.

In [2]:
data = pd.read_excel(DATA_PATH, sheet_name='2012')
parties = data['party']
votes_share = data.votes/sum(data.votes)
data['votes_per'] = votes_share
data.set_index('party', inplace = True)

In [3]:
data

,votes,votes_per,seats,diff
party,,,,
New Democracy,1825497,0.296565,129,21
Coalition of the Radical Left,1655022,0.268870,71,19
Panhellenic Socialist Movement,756024,0.122822,33,-8
Independent Greeks,462406,0.075121,20,-13
Popular Association-Golden Dawn,426025,0.069211,18,-3
Democratic Left,384986,0.062544,17,-2
Communist Party of Greece,277227,0.045038,12,-14
Recreate Greece-Action-Liberal Alliance,98140,0.015944,0,0
Popular Orthodox Rally,97099,0.015774,0,0


## Подсчёт ENV, ENS, DEV, RRP [задание 1]

Зададим функции, которые помогут подсчитать необходимые показатели

In [4]:
def ENV(voters_share: np.array or pd.Series):
    return 1/np.sum(voters_share**2)

def ENS(seats: np.array or pd.Series):
    return 1/np.sum((seats/sum(seats))**2)

def RRP(voters_share: np.array or pd.Series, seats: np.array or pd.Series):
    return (ENV(voters_share) - ENS(seats))/ENS(seats)

def DEV(voters_share: np.array or pd.Series, seats: np.array or pd.Series):
    return np.sum(abs(voters_share - seats/sum(seats)))

In [5]:
env = ENV(data.votes_per[parties])
ens = ENS(data.seats[parties])
dev = DEV(data.votes_per[parties], data.seats[parties])
rrp = RRP(data.votes_per[parties], data.seats[parties])

### Результат

In [6]:
print('ENV:', round(env, 2), '\nENS:', round(ens, 2), '\nDev(%):', round(dev*100, 2), '\nRRP(%):', round(rrp*100,2))

ENV: 5.2 
ENS: 3.76 
Dev(%): 26.69 
RRP(%): 38.23


## Партии, получившие места в парламенте

In [7]:
members = data[(data.seats > 0) & (data.seats < 300)].copy()

In [8]:
members

,votes,votes_per,seats,diff
party,,,,
New Democracy,1825497,0.296565,129,21
Coalition of the Radical Left,1655022,0.268870,71,19
Panhellenic Socialist Movement,756024,0.122822,33,-8
Independent Greeks,462406,0.075121,20,-13
Popular Association-Golden Dawn,426025,0.069211,18,-3
Democratic Left,384986,0.062544,17,-2
Communist Party of Greece,277227,0.045038,12,-14


## Предложение по голосованию [задание 2]

В этой части задания я внесу два предложения и покажу, как они поменяют Dev и RRP на имеющихся данных.

### Предложение 1:  
Распределять места не между теми партиями, которые набрали хотя бы 3% голосов, а опустить этот порог до 1%.

In [56]:
my_rule_members = data.loc[:, ['votes', 'votes_per', 'seats']].copy() 
my_rule_members['seats'] = np.round(my_rule_members[my_rule_members.votes_per > 0.01].votes/sum(my_rule_members[my_rule_members.votes_per > 0.01].votes)*(300-50))
my_rule_members.fillna(0, inplace = True)
my_rule_members.loc['New Democracy', 'seats'] = my_rule_members.loc['New Democracy', 'seats'] + 50
my_rule_members

,votes,votes_per,seats
party,,,
New Democracy,1825497,0.296565,126.0
Coalition of the Radical Left,1655022,0.268870,69.0
Panhellenic Socialist Movement,756024,0.122822,32.0
Independent Greeks,462406,0.075121,19.0
Popular Association-Golden Dawn,426025,0.069211,18.0
Democratic Left,384986,0.062544,16.0
Communist Party of Greece,277227,0.045038,12.0
Recreate Greece-Action-Liberal Alliance,98140,0.015944,4.0
Popular Orthodox Rally,97099,0.015774,4.0


In [57]:
dev_my = DEV(my_rule_members.votes_per[parties], my_rule_members.seats[parties])
rrp_my = RRP(my_rule_members.votes_per[parties], my_rule_members.seats[parties])

In [62]:
print('Если порог в 3% голосов заменить на 1%, то получим:\nDev(%):', round(dev_my*100, 2), '\nRRP(%):', round(rrp_my*100,2))

Если порог в 3% голосов заменить на 1%, то получим:
Dev(%): 24.69 
RRP(%): 31.59


### Предложение 2:  
В добавок к предыдущему предложению, партии, которая заняла первое место, давать не 50 дополнительных мест в парламенте, а 40.

In [59]:
my_rule_members = data.loc[:, ['votes', 'votes_per', 'seats']].copy() 
my_rule_members['seats'] = np.round(my_rule_members[my_rule_members.votes_per > 0.01].votes/sum(my_rule_members[my_rule_members.votes_per > 0.01].votes)*(300-40))
my_rule_members.fillna(0, inplace = True)
my_rule_members.loc['New Democracy', 'seats'] = my_rule_members.loc['New Democracy', 'seats'] + 40
my_rule_members

,votes,votes_per,seats
party,,,
New Democracy,1825497,0.296565,119.0
Coalition of the Radical Left,1655022,0.268870,72.0
Panhellenic Socialist Movement,756024,0.122822,33.0
Independent Greeks,462406,0.075121,20.0
Popular Association-Golden Dawn,426025,0.069211,19.0
Democratic Left,384986,0.062544,17.0
Communist Party of Greece,277227,0.045038,12.0
Recreate Greece-Action-Liberal Alliance,98140,0.015944,4.0
Popular Orthodox Rally,97099,0.015774,4.0


In [60]:
dev_my_40 = DEV(my_rule_members.votes_per[parties], my_rule_members.seats[parties])
rrp_my_40 = RRP(my_rule_members.votes_per[parties], my_rule_members.seats[parties])

In [63]:
print('Если порог прохождения: 1% голосов, и плюс к этому победителю добавляют не 50, а 40 мест, то получим:\nDev(%):', round(dev_my_40*100, 2), '\nRRP(%):', round(rrp_my_40*100,2))

Если порог прохождения: 1% голосов, и плюс к этому победителю добавляют не 50, а 40 мест, то получим:
Dev(%): 20.02 
RRP(%): 25.13


## Коалиции

Далее для упрощения работы я создам следующие списки: все коалиции (all_coalitions), выигрывающие коалиции (winning_coalitions), минимальные выигрывающие коалиции (minimal_winning_coalitions). В дальнейшем они необходимы для расчёта индексов.

### Все коалиции

In [9]:
all_coalitions = []
for i in range(1, len(members.index)+1):
    all_coalitions += [list(j) for j in combinations(members.index, i)]

### Выигрывающие коалиции

In [10]:
winning_coalitions = [coalition for coalition in all_coalitions if sum(members.seats[coalition]) > 150]

### Минимальные выигрывающие коалиции

In [11]:
minimal_winning_coalitions = []
for coalition in winning_coalitions:
    n = sum(members.seats[coalition])
    notmin = False
    for member in coalition:
        if n - members.seats[member] > 150:
            notmin = True
            break
    if not notmin:
        minimal_winning_coalitions.append(coalition) 

## Подсчёт индексов [задание 3]

Набор партий:

In [12]:
members.index

Index(['New Democracy', 'Coalition of the Radical Left',
       'Panhellenic Socialist Movement', 'Independent Greeks',
       'Popular Association-Golden Dawn', 'Democratic Left',
       'Communist Party of Greece'],
      dtype='object', name='party')

### Индекс Банцафа

In [13]:
#Находим для каждой партии число коалиций, в которых она является ключевой
b = np.array([])
for member in members.index:
    b_member = 0
    for coalition in winning_coalitions:
        if (member in coalition) and (sum(members.seats[coalition]) - members.seats[member] <= 150):
            b_member += 1
    b = np.append(b, b_member)

#Считаем индекс Банцафа
bancaf = b/sum(b)
bancaf

array([0.55102041, 0.10204082, 0.10204082, 0.06122449, 0.06122449,
       0.06122449, 0.06122449])

### Индекс Шепли-Шубика

In [14]:
#Считаем индекс Шепли-Шубика
phi = np.array([])
for member in members.index:
    phi_member = 0
    for coalition in winning_coalitions:
        if (member in coalition) and (sum(members.seats[coalition]) - members.seats[member] <= 150):
            phi_member += m.factorial(len(members.index) - len(coalition))*m.factorial(len(coalition)-1)\
                        /m.factorial(len(members.index))
    phi = np.append(phi, phi_member)

phi

array([0.52380952, 0.12380952, 0.12380952, 0.05714286, 0.05714286,
       0.05714286, 0.05714286])

### Общий индекс Джонсона

In [15]:
key_members_in_winning_coalitions = []
for coalition in winning_coalitions:
    key_members = 0
    for member in coalition:
        if sum(members.seats[coalition]) - members.seats[member] <= 150:
            key_members += 1
    key_members_in_winning_coalitions.append(key_members)

In [16]:
TJI = np.array([])
for member in members.index:
    TJI_member = 0
    for i in range(len(winning_coalitions)):
        if (member in winning_coalitions[i]) and (sum(members.seats[winning_coalitions[i]]) - members.seats[member] <= 150):
            TJI_member += 1/key_members_in_winning_coalitions[i]
    TJI = np.append(TJI, TJI_member)
TJI 

array([45. ,  3.8,  3.8,  1.6,  1.6,  1.6,  1.6])

### Индекс влияния конкретной партии Джонсона

На основе рассчитанного ранее общего индекса Джонсона, рассчитываем индекс влияния конкретной партии Джонсона.

In [17]:
JI = TJI/sum(TJI)
JI

array([0.76271186, 0.06440678, 0.06440678, 0.02711864, 0.02711864,
       0.02711864, 0.02711864])

### Общий индекс Дигена – Пакела

In [18]:
TDPI = np.array([])
for member in members.index:
    TDPI_member = 0
    for coalition in minimal_winning_coalitions:
        if member in coalition:
            TDPI_member += 1/len(coalition)
    TDPI = np.append(TDPI, TDPI_member)
TDPI

array([3. , 1.3, 1.3, 1.6, 1.6, 1.6, 1.6])

### Индекс влияния конкретной партии Дигена – Пакела

На основе рассчитанного ранее общего индекса Дигена – Пакела, рассчитываем индекс влияния конкретной партии Дигена – Пакела.

In [19]:
DPI = TDPI/sum(TDPI)
DPI

array([0.25      , 0.10833333, 0.10833333, 0.13333333, 0.13333333,
       0.13333333, 0.13333333])

### Индекс Холера – Пакела

In [20]:
h = np.array([])
for member in members.index:
    h_member = 0
    for coalition in minimal_winning_coalitions:
        if member in coalition:
            h_member += 1
    h = np.append(h, h_member)

HPI = h/sum(h)
HPI

array([0.19047619, 0.11904762, 0.11904762, 0.14285714, 0.14285714,
       0.14285714, 0.14285714])

### Таблица с подсчитанными индексами

In [21]:
indices_df = pd.DataFrame({'Banzhaf': bancaf, 'Shapley–Shubik': phi, 'TJI': TJI, 'JI': JI, 'TDPI': TDPI, 'DPI': DPI, 'HPI': HPI}, index = members.index)
indices_df

,Banzhaf,Shapley–Shubik,TJI,JI,TDPI,DPI,HPI
party,,,,,,,
New Democracy,0.551020,0.523810,45.0,0.762712,3.0,0.250000,0.190476
Coalition of the Radical Left,0.102041,0.123810,3.8,0.064407,1.3,0.108333,0.119048
Panhellenic Socialist Movement,0.102041,0.123810,3.8,0.064407,1.3,0.108333,0.119048
Independent Greeks,0.061224,0.057143,1.6,0.027119,1.6,0.133333,0.142857
Popular Association-Golden Dawn,0.061224,0.057143,1.6,0.027119,1.6,0.133333,0.142857
Democratic Left,0.061224,0.057143,1.6,0.027119,1.6,0.133333,0.142857
Communist Party of Greece,0.061224,0.057143,1.6,0.027119,1.6,0.133333,0.142857
